# Jaringan - LKS
 Praktek NLP

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
import matplotlib.pyplot as plt




# Untuk mempermudah, simpan setiap objek agar dapat digunakan untuk pemodelan maupun deployment. Gunakan library Pickle
import pickle


%matplotlib inline

# 01 Data Acquisition


In [4]:
data = pd.read_csv('jaringanClean.csv')
# data.drop(['text'],axis = 1,inplace = True)
data.rename(columns={'sentiment': 'Sentiment'}, inplace=True)
data = data.head(5000)
data

,Sentiment,text,clean_teks
0,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan Produk Bangsa Sendiri <PROVIDER_NAME>,produk bangsa
1,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDER_NAME>",sakti alhamdulillah v
2,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenapa di dalam kamar sinyal 4G hilang yang 1 lagi panggilan darurat saja <URL>,selamat pagi bantu kamar sinyal hilang panggil darurat
3,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data lemot banget padahal H+ !!!!,dear jaring data lot banget h
4,negative,Selamat malam PENDUSTA <PROVIDER_NAME>,selamat malam dusta
...,...,...,...
295,positive,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yang 4G lancar sekali di lombok timur",pantesan lancar ya sinyal lancar lombok timur
296,positive,Alhamdulillah lancar pakai <PROVIDER_NAME>,alhamdulillah lancar pakai
297,positive,"Untung pakai internet <PROVIDER_NAME>, lancar, jadi dapat banyak ide.",untung pakai internet lancar ide
298,positive,"di tempat ramai seperti di lokasi wisata, <PROVIDER_NAME> tetap jaya.",ramai lokasi wisata jaya


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   300 non-null    object
 1   text        300 non-null    object
 2   clean_teks  297 non-null    object
dtypes: object(3)
memory usage: 7.2+ KB


In [6]:

data.dropna(inplace = True)
data.isnull().sum()

Sentiment     0
text          0
clean_teks    0
dtype: int64

In [7]:
print('Total Jumlah Sentimen:', data.shape[0], 'data\n')
print('terdiri dari (sentimen):')
print('-- [0] Positive\t:', data[data.Sentiment == 'positive'].shape[0], 'data')
print('-- [1] Negative\t:', data[data.Sentiment == 'negative'].shape[0], 'data')


Total Jumlah Sentimen: 297 data

terdiri dari (sentimen):
-- [0] Positive	: 137 data
-- [1] Negative	: 160 data


In [8]:
# data['Sentiment'] = data['Sentiment'].replace({'positive': 1, 'negative': 2})


# 03 Feature Engineering

In [9]:
# Pisahkan kolom fitur dan target
X = data['clean_teks']
y = data['Sentiment']

In [10]:
X

0                                               produk bangsa
1                                       sakti alhamdulillah v
2      selamat pagi bantu kamar sinyal hilang panggil darurat
3                               dear jaring data lot banget h
4                                         selamat malam dusta
                                ...                          
295             pantesan lancar ya sinyal lancar lombok timur
296                                alhamdulillah lancar pakai
297                          untung pakai internet lancar ide
298                                  ramai lokasi wisata jaya
299                           sinyal amsyong stadion gajayana
Name: clean_teks, Length: 297, dtype: object

In [11]:
y

0      positive
1      positive
2      negative
3      negative
4      negative
         ...   
295    positive
296    positive
297    positive
298    positive
299    negative
Name: Sentiment, Length: 297, dtype: object

In [ ]:
import pandas as pd
import numpy as np
from math import log

# Langkah 1: Membaca data dari Excel
texts = X.astype(str).tolist()  # Sesuaikan dengan nama kolom teks

# Langkah 2: Membangun kosakata dan menghitung TF
def compute_tf(texts):
    vocab = set(word for text in texts for word in text.split())
    tf_scores = []
    for text in texts:
        words = text.split()
        word_count = len(words)
        tf_text = {}
        for word in words:
            if word in vocab:
                tf_text[word] = tf_text.get(word, 0) + 1/word_count
        tf_scores.append(tf_text)
    return vocab, tf_scores

vocab, tf_scores = compute_tf(texts)

# Langkah 3: Menghitung IDF
def compute_idf(vocab, texts):
    n_docs = len(texts)
    idf_scores = {}
    for word in vocab:
        n_containing = sum(1 for text in texts if word in text)
        idf_scores[word] = log(n_docs / (1 + n_containing))  # Ditambahkan 1 untuk mencegah pembagian dengan nol
    return idf_scores

idf_scores = compute_idf(vocab, texts)

# Langkah 4: Menghitung skor TF-IDF
def compute_tfidf(tf_scores, idf_scores):
    tfidf_scores = []
    for tf_text in tf_scores:
        tfidf_text = {word: tf * idf_scores[word] for word, tf in tf_text.items()}
        tfidf_scores.append(tfidf_text)
    return tfidf_scores

tfidf_scores = compute_tfidf(tf_scores, idf_scores)

# Hasil akhir: tfidf_scores adalah daftar dari dictionary yang berisi skor TF-IDF untuk setiap dokumen


In [12]:
import pandas as pd
from TFIDF import TFIDFVectorizer

# Contoh data teks (sudah didefinisikan sebelumnya dalam X)


# Menggunakan TFIDFVectorizer yang telah kita buat sebelumnya
vectorizer_sample = TFIDFVectorizer()
vectorizer_sample.fit(X)
X_tfidf_sample = vectorizer_sample.transform(X)

# Mengubah hasil ke dalam DataFrame
feature_names = vectorizer_sample.get_feature_names_out()
data_tf_idf_sample = pd.DataFrame(X_tfidf_sample, columns=feature_names)

# Menampilkan 100 baris pertama (dalam kasus ini, seluruh dataset karena kurang dari 100)
data_tf_idf_sample.head()



,acara,aceh,adhan,ajaib,ajar,akses,aksi,aktif,aktivitas,alat,...,wkp,wow,xd,ya,yess,yonder,youtube,youtubenya,yuk,zalim
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
data_tf_idf_sample.to_csv('data_tf_idf.csv', index=False)

In [ ]:
pd.Series(feature_names).to_csv('feature_names.csv', index=False)


In [13]:
data_tf_idf_sample.shape

(297, 621)

In [14]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(data_tf_idf_sample, y, test_size=0.2, random_state=40)

In [15]:
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [16]:
def train_test_split_corrected(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    
    # Mengacak index
    shuffled_indices = np.random.permutation(len(X))
    
    # Menentukan ukuran test set
    test_set_size = int(len(X) * test_size)
    
    # Memisahkan indeks untuk test dan train
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    
    # Pemilihan data berdasarkan indeks untuk X dan y
    X_train = X.iloc[train_indices]
    X_test = X.iloc[test_indices]
    
    if isinstance(y, pd.Series):
        y_train = y.iloc[train_indices]
        y_test = y.iloc[test_indices]
    else:
        y_train = y[train_indices]
        y_test = y[test_indices]
    
    return X_train, X_test, y_train, y_test

In [17]:

X_train, X_test, y_train, y_test = train_test_split_corrected(data_tf_idf_sample, y, test_size=0.2, random_state=40)

X_train.shape, X_test.shape, y_train.shape, y_test.shape



((238, 621), (59, 621), (238,), (59,))

In [18]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

X_train = X_train.values
X_test = X_test.values



In [19]:
from KNN import KNN


clf = KNN(k=3)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

print(predictions)

acc = np.sum(predictions == y_test) / len(y_test)
print(acc)

['negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative']
0.7457627118644068


In [20]:
predictions = np.array(predictions)
y_test = np.array(y_test)

# Hitung jumlah prediksi yang benar dan yang salah
prediksi_benar = (predictions == y_test).sum()
prediksi_salah = (predictions != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

# Hitung akurasi
accuracy = prediksi_benar / (prediksi_benar + prediksi_salah) * 100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 44
Jumlah prediksi salah	: 15
Akurasi pengujian	: 74.57627118644068 %


# Not Scratch


In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier      # Pilih salah satu algoritma supervised learning. Contoh ini menggunakan algoritma Decision Tree
from joblib import dump   


# Define the KNN classifier
knn = KNeighborsClassifier()

# Define the grid of parameters to search
param_grid = {'n_neighbors': np.arange(1,100)}  # Example values for n_neighbors

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best model after tuning
best_model = grid_search.best_estimator_

# Save the best model
dump(best_model, 'best_knn_model.joblib')

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

n_neighbors = best_params['n_neighbors']

algorithm = KNeighborsClassifier(n_neighbors=n_neighbors)               # Load algoritma pembelajaran
model = algorithm.fit(X_train, y_train)   # Fitkan (latih) algoritma pada data latih & label latih

dump(model, filename='model_1.joblib')

Best parameters: {'n_neighbors': 4}


['model_1.joblib']

In [22]:
model_pred = model.predict(X_test)

In [23]:
# Hitung jumlah data yang berhasil di prediksi model & jumlah data yang salah di prediksi
prediksi_benar = (model_pred == y_test).sum()
prediksi_salah = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

accuracy = prediksi_benar / (prediksi_benar + prediksi_salah)*100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 44
Jumlah prediksi salah	: 15
Akurasi pengujian	: 74.57627118644068 %
